# Block

## Basic Usage

A block is a piece of parameterized circuit.
Blocks are listed in self.block_list to form a circuit.

In [3]:
from Utilities.CircuitEvaluation import get_quantum_engine
from projectq.ops import All, Measure
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction

from Blocks import RotationEntangler

print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
entangler=RotationEntangler((1,2,3),(3,2,1))
print("The entangler to apply:")
print(entangler)
entangler.apply([0.5],wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
entangler.is_inversed=True
print(entangler)
entangler.apply([0.5],wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))

All(Measure) | wavefunction
compiler_engine.flush()


The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
The amplitude of |0000>: (0.8775825618903725+0j)
Type:RotationEntangler; Para Num:1; INVERSED; Qsubset:(1, 2, 3); Pauli:ZYX
The amplitude of |0000>: (0.9999999999999997+0j)


## Provided Blocks

We tends to call a block *entangler* if it can entangle the qubits it acts on. （参考源代码完成下面 胡）

- Rotation Entangler $e^{iPt}$
- Multi-Rotation Entangler $e^{iP_1 t_1} e^{iP_2 t_2} .. e^{iP_n t_n}$
- Single Parameter Multi-Rotation Entangler $e^{i P_1 a_1 t} e^{iP_2 a_2 t} .. e^{iP_n a_n t}$
- Efficient Coupled Cluster
- Hardware Efficient Entangler
- Hartree-Fock Initial Block
- Pauli Gates Block
- Time Evolution Block
- Compositive Block



### Entangler blocks
### hardware efficient entangler
### Pauli entangler

### Time evolution block

### Initial block